In [1]:
import os

In [2]:
pwd

'/Users/hitam/Downloads/TextSummarizer/research'

In [3]:
os.chdir('../')

In [4]:
pwd

'/Users/hitam/Downloads/TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [6]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/Users/hitam/Downloads/TextSummarizer/texts/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        train_dataset = dataset_samsum_pt["test"]
        eval_dataset = dataset_samsum_pt["validation"]



        #trainer_args = TrainingArguments(
          #  output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
         #   per_device_train_batch_size=1, per_device_eval_batch_size=1,
          #  weight_decay=0.01, logging_steps=10,
         #   evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
          #  gradient_accumulation_steps=16
        #) 
       

        # Define Trainer arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )

        # Start training
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2023-12-12 23:15:07,100: INFO: common: Yaml File : config/config.yaml loaded successfully]
[2023-12-12 23:15:07,133: INFO: common: Yaml File : params.yaml loaded successfully]
[2023-12-12 23:15:07,135: INFO: common: created directory at path: artifacts]
[2023-12-12 23:15:07,139: INFO: common: created directory at path: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|█▉        | 10/51 [57:15<4:16:25, 375.26s/it]

{'loss': 3.1101, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [1:55:39<2:49:01, 327.15s/it]

{'loss': 3.0467, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

 98%|█████████▊| 50/51 [4:09:54<04:08, 248.06s/it]

{'loss': 2.8544, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [4:13:56<00:00, 246.40s/it]

{'train_runtime': 15236.8569, 'train_samples_per_second': 0.054, 'train_steps_per_second': 0.003, 'train_loss': 3.0233091560064578, 'epoch': 1.0}


100%|██████████| 51/51 [4:13:57<00:00, 298.77s/it]
